In [2]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.printSchema()

providing ../data/raw/2022_place_canvas_history-000000000050.csv ...
../data/raw/2022_place_canvas_history-000000000050.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0fb09a32-b101-4b4b-9d52-dba7cea45f82;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 75ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

root
 |-- user_id: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)



<div style="color:lightgreen">
Eine zweite zu betrachtende Situation ist, dass Moderatoren auch mehrere Pixel auf einmal ändern können.

```
SELECT user_id, COUNT(x) --es reicht aus die x Koordinate zu zählen
FROM dataFrame
WHERE --Wie könnte die Where-Clause angegeben werden, dass nur ein bestimmter Zeitpunkt betrachtet wird?
HAVING COUNT(x) > 1
GROUP BY user_id;
```


Vorschlag Sebastian:
<div style="color:lightblue">(Sebastian): Gruppieren nach t-Spalte und dann gucken wo count(t) > 1 ?</div>
</div>

<div style="color:lightgreen">

Mein Vorschlag:

Ist es nicht besser direkt nach Nutzer und Zeit zu gruppieren? Dann hat man für jeden Nutzer direkt angezeigt, wie oft zu einem Zeitpunkt Pixel platziert wurden?
Dann wäre auch die Bereinigung der Daten effektiver als oben.

Hier der Versuch:
</div>

In [3]:
from src.features.feature_functions import group_dataframe_by_time_and_user

timeUserGrouped = group_dataframe_by_time_and_user(dataFrame)
#timeUserGrouped = group_dataframe_by_columns(dataFrame, ['t', 'user_id'])
timeUserGrouped.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- t: long (nullable = true)
 |-- count: long (nullable = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [4]:
timeUserGrouped = timeUserGrouped.withColumnRenamed('count','time_count')

timeUserGrouped = timeUserGrouped.filter(timeUserGrouped.time_count > 1)
timeUserGrouped.show()

+--------------------+----+----------+--------------------+--------------------+-------------------------+
|             user_id|   t|time_count|     collect_list(x)|     collect_list(y)|collect_list(pixel_color)|
+--------------------+----+----------+--------------------+--------------------+-------------------------+
|X8gpDfO6+Z87SKKTy...|2974|         8|[1685, 904, 932, ...|[228, 1865, 1850,...|     [#FFD635, #000000...|
|AxBTmhBMbr1sbDw/2...|2344|         6|[1403, 777, 1741,...|[2, 8, 10, 2, 8, 10]|     [#FFA800, #FFA800...|
|DIECtVqoJqBlL6A0C...|1200|         6|[932, 920, 933, 9...|[1757, 1832, 1838...|     [#FFFFFF, #BE0039...|
|AxBTmhBMbr1sbDw/2...|1726|         6|[655, 612, 1575, ...|[1896, 1917, 7, 1...|     [#B44AC0, #E4ABFF...|
|LIVvK+L7Y5WOXCwhH...|2983|         4|[542, 886, 542, 886]|[1942, 10, 1942, 10]|     [#000000, #FFA800...|
|MsdvGyv1KSfWjE8OI...|1444|         4|[957, 954, 957, 954]|[1844, 1844, 1844...|     [#FFA800, #FFA800...|
|x1FP8fj5v0RvrYdDE...| 436|         4

In [5]:
row = dataFrame.select('user_id').distinct().count()
print(f'Anzahl der herkömmlichen Zeilen: {row}')

row_sel = timeUserGrouped.select('user_id').distinct().count()
print(f'Anzahl der angeblichen Moderatoren: {row_sel}')

Anzahl der herkömmlichen Zeilen: 733458


22/06/05 15:00:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 15:00:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 15:00:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 15:00:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 15:00:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 15:00:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 15:00:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 15:00:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 15:00:24 WARN RowBasedKeyValueBatch: Calling spill() on

Anzahl der angeblichen Moderatoren: 733458
